<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
<font size='1'>Basado en material de Karim Pichara y Christian Pieringer. Todos los derechos reservados.</font>
</p>

<h1>Multiherencia</h1>

Tal como es posible que una subclase herede datos y comportamiento de una superclase, también es posible heredar
de más de una clase a la vez:

In [ ]:
class Investigador:
    def __init__(self, area):
        self.area = area
        
class Docente:
    def __init__(self, departamento):
        self.departamento = departamento
        
class Academico(Docente, Investigador):
    def __init__(self, nombre, area_investigacion, departamento):
        #esto no es del todo correcto, coming soon...
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre

p1 = Academico("Juan Perez", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

<h2> Multiherencia y el Problema del diamante</h2>

El siguiente ejemplo muestra lo que ocurre en un contexto de multiherencia si es que cada sub-clase llama directamente a inicializar a todas sus superclases. La figura siguiente muestra la jerarquía de las clases en cuestión

![Diamante](figs/img_diamante.png)

El siguiente código muestra qué ocurre cuando llamamos al método `llamar()` en ambas super clases desde la clase `SubClaseA`.

In [ ]:
class ClaseB:
    num_llamadas_B = 0
    def llamar(self):
        print("Llamando método en Clase B")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    num_llamadas_izq = 0
    def llamar(self):
        ClaseB.llamar(self)
        print("Llamando método en Subclase izquierda")
        self.num_llamadas_izq += 1

class SubClaseDerecha(ClaseB):
    num_llamadas_der = 0
    def llamar(self):
        ClaseB.llamar(self)
        print("Llamando método en Subclase derecha")
        self.num_llamadas_der += 1

class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    num_llamadas_subA = 0
    def llamar(self):
        SubClaseIzquierda.llamar(self)
        SubClaseDerecha.llamar(self)
        print("Llamando método en SubclaseA")
        self.num_llamadas_subA += 1

s = SubClaseA()
s.llamar()
print(s.num_llamadas_subA, s.num_llamadas_izq, s.num_llamadas_der, s.num_llamadas_B)

Del output se puede apreciar que la clase de más arriba en la jerarquía ("Clase B"), fue llamada dos veces, a pesar de que el objetivo era llamarla sólo una vez. La estructura de jerarquía en forma de diamante ocurre siempre que tengamos una clase que hereda de dos clases. Ya que como en Python todo es un objeto, todo hereda de la clase "object" (ver: "new style classes (https://www.python.org/doc/newstyle/)"), por lo tanto en general el esquema de multiherencia se ve de la siguiente forma:

![Diamante2](figs/img_diamante_2.png)

Siguiendo el mismo ejemplo anterior, en vez de llamar al método "llamar()", llamamos al método "__init__", estaríamos inicializando dos veces en la clase "object"!!. 

<h2>Solución:</h2>

La solución es que cada clase debe preocuparse de llamar a inicializar a la clase que la "precede" en el orden del esquema de la multiherencia. En Python el orden de las clases va de izquierda a derecha dentro de la lista de super-clases desde donde hereda la sub-clase. En este caso, simplemente debemos preocuparnos de hacer una llamada a "super()", Python se encargará de que la llamada corresponda a la clase que respeta el orden en la multiherencia, en este caso, después de la subclase viene la clase "SubclaseIzquierda", después "SubClaseDerecha" y finalmente "ClaseB"

In [ ]:
class ClaseB:
    num_llamadas_B = 0
    def llamar(self):
        print("Llamando método en Clase B")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    num_llamadas_izq = 0
    def llamar(self):
        super().llamar()
        print("Llamando método en Subclase Izquierda")
        self.num_llamadas_izq += 1

class SubClaseDerecha(ClaseB):
    num_llamadas_der = 0
    def llamar(self):
        super().llamar()
        print("Llamando método en Subclase Derecha")
        self.num_llamadas_der += 1

class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    num_llamadas_subA = 0
    def llamar(self):
        super().llamar()
        print("Llamando método en SubclaseA")
        self.num_llamadas_subA += 1

s = SubClaseA()
s.llamar()
print(s.num_llamadas_subA, s.num_llamadas_izq, s.num_llamadas_der, s.num_llamadas_B)

### El método `__mro__` (method resolution order) nos muestra el orden de la jerarquía. 

Para casos de multiherencia más complejos, Python utiliza el algoritmo C3 para calcular un orden lineal entre las clases que participan en el esquema de multiherencia: 


In [ ]:
SubClaseA.__mro__

Esto significa que cualquier estructura de multiherencia que no pueda ser resuelta por el algoritmo C3, no está permitida en Python:

In [ ]:
class X():
    def call_me(self):
        print("soy X")
    
class Y():
    def call_me(self):
        print("soy Y")
    
class A(X, Y):
    def call_me(self):
        print("soy A")
    
class B(Y, X):
     def call_me(self):
         print("soy B")

class F(A, B):
    def call_me(self):
        print("soy F")